In [1]:
from datetime import datetime
import pyarrow as py
import pandas as pd
import numpy as np
import time
import pytz


df_table_merged_unprocessed = pd.read_csv("../Source/Data_processed/Table_merged.csv", sep=";")

In [2]:
df_table_merged_unprocessed.head()

,id_prod,date,session_id,client_id,price,categ,sex,birth
0,0_0,2021-03-01 13:45:51.575117,s_282,c_5152,3.75,0,f,1986.0
1,0_0,2021-03-02 06:42:55.351333,s_621,c_2917,3.75,0,m,1988.0
2,0_0,2021-03-02 18:49:49.651862,s_852,c_3988,3.75,0,f,1962.0
3,0_0,2021-03-02 21:57:33.862118,s_908,c_1004,3.75,0,m,1973.0
4,0_0,2021-03-03 23:59:48.997483,s_1379,c_278,3.75,0,f,1987.0


In [3]:
def split_column_using_space(df, new_col1, new_col2, target_col):
    df[[new_col2, new_col1]] = df[target_col].str.split(expand=True)
    return df

split_column_using_space(df_table_merged_unprocessed, 'Time', 'Date', 'date')

,id_prod,date,session_id,client_id,price,categ,sex,birth,Date,Time
0,0_0,2021-03-01 13:45:51.575117,s_282,c_5152,3.75,0,f,1986.0,2021-03-01,13:45:51.575117
1,0_0,2021-03-02 06:42:55.351333,s_621,c_2917,3.75,0,m,1988.0,2021-03-02,06:42:55.351333
2,0_0,2021-03-02 18:49:49.651862,s_852,c_3988,3.75,0,f,1962.0,2021-03-02,18:49:49.651862
3,0_0,2021-03-02 21:57:33.862118,s_908,c_1004,3.75,0,m,1973.0,2021-03-02,21:57:33.862118
4,0_0,2021-03-03 23:59:48.997483,s_1379,c_278,3.75,0,f,1987.0,2021-03-03,23:59:48.997483
...,...,...,...,...,...,...,...,...,...,...
687550,2_99,2022-05-21 01:53:46.967570,s_211661,c_5828,84.99,2,f,1998.0,2022-05-21,01:53:46.967570
687551,2_99,2022-07-21 01:53:46.967570,s_241374,c_5828,84.99,2,f,1998.0,2022-07-21,01:53:46.967570
687552,2_99,2022-11-11 20:56:49.820935,s_296127,c_7135,84.99,2,m,1996.0,2022-11-11,20:56:49.820935
687553,2_99,2022-12-11 20:56:49.820935,s_310606,c_7135,84.99,2,m,1996.0,2022-12-11,20:56:49.820935


In [4]:
def convert_column_to_datetime(df, column_name):
    df[column_name] = pd.to_datetime(df[column_name],errors='coerce')
    return df
convert_column_to_datetime(df_table_merged_unprocessed, 'date',)

,id_prod,date,session_id,client_id,price,categ,sex,birth,Date,Time
0,0_0,2021-03-01 13:45:51.575117,s_282,c_5152,3.75,0,f,1986.0,2021-03-01,13:45:51.575117
1,0_0,2021-03-02 06:42:55.351333,s_621,c_2917,3.75,0,m,1988.0,2021-03-02,06:42:55.351333
2,0_0,2021-03-02 18:49:49.651862,s_852,c_3988,3.75,0,f,1962.0,2021-03-02,18:49:49.651862
3,0_0,2021-03-02 21:57:33.862118,s_908,c_1004,3.75,0,m,1973.0,2021-03-02,21:57:33.862118
4,0_0,2021-03-03 23:59:48.997483,s_1379,c_278,3.75,0,f,1987.0,2021-03-03,23:59:48.997483
...,...,...,...,...,...,...,...,...,...,...
687550,2_99,2022-05-21 01:53:46.967570,s_211661,c_5828,84.99,2,f,1998.0,2022-05-21,01:53:46.967570
687551,2_99,2022-07-21 01:53:46.967570,s_241374,c_5828,84.99,2,f,1998.0,2022-07-21,01:53:46.967570
687552,2_99,2022-11-11 20:56:49.820935,s_296127,c_7135,84.99,2,m,1996.0,2022-11-11,20:56:49.820935
687553,2_99,2022-12-11 20:56:49.820935,s_310606,c_7135,84.99,2,m,1996.0,2022-12-11,20:56:49.820935


In [5]:
from datetime import datetime

def clean_table_merged(table_merged_unprocessed):
    table_merged_unprocessed['Time'] = table_merged_unprocessed['Time'].str.replace("24:", "00:", regex=False)
    table_merged_unprocessed['Timestamp'] = table_merged_unprocessed['Date'] + ' ' + table_merged_unprocessed['Time']
    table_merged_unprocessed = table_merged_unprocessed.drop(columns=['date','Date','Time'])
    return table_merged_unprocessed

table_merged_processed = clean_table_merged(df_table_merged_unprocessed.copy())
table_merged_processed.head()

,id_prod,session_id,client_id,price,categ,sex,birth,Timestamp
0,0_0,s_282,c_5152,3.75,0,f,1986.0,2021-03-01 13:45:51.575117
1,0_0,s_621,c_2917,3.75,0,m,1988.0,2021-03-02 06:42:55.351333
2,0_0,s_852,c_3988,3.75,0,f,1962.0,2021-03-02 18:49:49.651862
3,0_0,s_908,c_1004,3.75,0,m,1973.0,2021-03-02 21:57:33.862118
4,0_0,s_1379,c_278,3.75,0,f,1987.0,2021-03-03 23:59:48.997483


In [6]:
table_merged_processed = convert_column_to_datetime(table_merged_processed, 'Timestamp')
table_merged_processed

,id_prod,session_id,client_id,price,categ,sex,birth,Timestamp
0,0_0,s_282,c_5152,3.75,0,f,1986.0,2021-03-01 13:45:51.575117
1,0_0,s_621,c_2917,3.75,0,m,1988.0,2021-03-02 06:42:55.351333
2,0_0,s_852,c_3988,3.75,0,f,1962.0,2021-03-02 18:49:49.651862
3,0_0,s_908,c_1004,3.75,0,m,1973.0,2021-03-02 21:57:33.862118
4,0_0,s_1379,c_278,3.75,0,f,1987.0,2021-03-03 23:59:48.997483
...,...,...,...,...,...,...,...,...
687550,2_99,s_211661,c_5828,84.99,2,f,1998.0,2022-05-21 01:53:46.967570
687551,2_99,s_241374,c_5828,84.99,2,f,1998.0,2022-07-21 01:53:46.967570
687552,2_99,s_296127,c_7135,84.99,2,m,1996.0,2022-11-11 20:56:49.820935
687553,2_99,s_310606,c_7135,84.99,2,m,1996.0,2022-12-11 20:56:49.820935


In [7]:
table_merged_processed.to_csv('../Source/Data_processed/DateTime_cleaned_temp.csv', sep=';', index=False)

In [8]:
final_df = pd.read_csv('../Source/Data_processed/DateTime_cleaned.csv', sep=";", in)

SyntaxError: invalid syntax (3546022581.py, line 1)